# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [59]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [49]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [50]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'met', 'uk_food']


In [51]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [52]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [53]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Leas',
 'AddressLine2': 'Kingsdown',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Walmer and Kingsdown Golf Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.793855801259,
 'FHRSID': 452092,
 'LocalAuthorityBusinessID': 'PI/000040614',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT14 8EP',
 'RatingDate': '2019-09-10T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65737bf87334ea1c735f22ed'),
 'geocode': {'latitude': '51.1787300109863', 'longitude': '1.40325689315796'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/452092',
            'rel': 'self'}],
 'longitude': None,
 'meta': {'d

In [54]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [55]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [56]:
# Insert the new restaurant into the collection
db.establishments.insert_one(new_restaurant)

In [57]:
# Check that the new restaurant was inserted
db.establishments.find_one(new_restaurant)

{'_id': ObjectId('6579483865fc072c1ba87069'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [58]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = establishments.find({"BusinessType": "Restaurant/Cafe/Canteen"},{"BusinessTypeID": 1})
results = establishments.find(query)
for result in results:
    print(result)

TypeError: filter must be an instance of dict, bson.son.SON, or any other type that inherits from collections.Mapping

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
document = establishments.find_one({"name": "New Restaurant"})


establishments.update_one(
    {"_id": document["_id"]},
    {"$set": {"BusinessTypeID": ""}}
)

In [ ]:
# Confirm that the new restaurant was updated
db.establishments.find_one(new_restaurant)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [31]:
# Find how many documents have LocalAuthorityName as "Dover"
Dover_documents = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(Dover_documents)

994


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
deleted_dover_documents = establishments.delete_many({"LocalAuthorityName": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
Dover_documents = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(Dover_documents)

In [ ]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [38]:
#Change the data type from String to Decimal for longitude and latitude
results = establishments.update_many({}, [
    {"$set": {"longitude": {"$toDouble": "$longitude"}}},
    {"$set": {"latitude": {"$toDouble": "$latitude"}}}
])

Use `update_many` to convert `RatingValue` to integer numbers.

In [39]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [40]:
# Change the data type from String to Integer for RatingValue
results = establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])

In [47]:
# Check that the coordinates and rating value are now numbers
longitude = establishments.find({"longitude": {"$type": 1}})
pprint(longitude)
latitude = establishments.find({"latitude": {"$type": 1}})
pprint(latitude)
rating_value = establishments.find({"RatingValue": {"$type": 1}})
pprint(rating_value)